This is a temporary set of tests.

In [1]:
from quantumgraph import QuantumGraph, ExpectationValue
import numpy as np

An IBMQ account could not be loaded


Set a tolerance for how close the numbers generated by `QuantumGraph` should be to the correct values.

In [2]:
eps = 0.8  # Tolerance for numerical errors

Whether to test using expectation value simulator.

In [3]:
test_ev = False

Tests are done on a graph of 5 qubits

In [5]:
n = 5  # Number of qubits

if test_ev:
    device = ExpectationValue(n)
    graph = QuantumGraph(n,device=device)
else:
    graph = QuantumGraph(n)

First we check that all qubits are indeed in the $\langle Z \rangle = 1$ state.

In [6]:
initial_rho = {'X': 0.0, 'Y': 0.0, 'Z': 1.0}
for j in range(n):
    print(f"Qubit {j} Bloch vector: {graph.get_bloch(j)}")
    rho = graph.get_bloch(j)
    for pauli in ['X','Y','Z']:
        assert abs(rho[pauli]-initial_rho[pauli])<eps, print('Problem with qubit',j,':',rho)
        print(f"Qubit {j} - Pauli {pauli}: {rho[pauli]}")

Qubit 0 Bloch vector: {'X': np.float64(0.0020904066266182325), 'Y': np.float64(-2.210936702413843e-06), 'Z': np.float64(0.9932157860761144)}
Qubit 0 - Pauli X: 0.0020904066266182325
Qubit 0 - Pauli Y: -2.210936702413843e-06
Qubit 0 - Pauli Z: 0.9932157860761144
Qubit 1 Bloch vector: {'X': np.float64(-0.003354575016331754), 'Y': np.float64(-0.0013138060984100143), 'Z': np.float64(0.993022648462849)}
Qubit 1 - Pauli X: -0.003354575016331754
Qubit 1 - Pauli Y: -0.0013138060984100143
Qubit 1 - Pauli Z: 0.993022648462849
Qubit 2 Bloch vector: {'X': np.float64(-0.00164962398513394), 'Y': np.float64(0.0018514107167697641), 'Z': np.float64(0.9941432756951121)}
Qubit 2 - Pauli X: -0.00164962398513394
Qubit 2 - Pauli Y: 0.0018514107167697641
Qubit 2 - Pauli Z: 0.9941432756951121
Qubit 3 Bloch vector: {'X': np.float64(0.0013631589217876895), 'Y': np.float64(-0.0007352744869711403), 'Z': np.float64(0.9929969330502922)}
Qubit 3 - Pauli X: 0.0013631589217876895
Qubit 3 - Pauli Y: -0.0007352744869711

Now set qubit 2 to the $\langle X \rangle = 1$ state, check that it works and also that the rest are still in the $\langle Z \rangle = 1$ state.

In [7]:
rhos = {}
rho_2 = {'X': 1.0}
graph.set_bloch(rho_2,2)
print(f"Qubit 2 Bloch vector after set_bloch: {graph.get_bloch(2)}")
for j in range(n):
    rho = graph.get_bloch(j)
    for pauli in ['X','Y','Z']:
        if j==2:
            assert abs(rho[pauli]-rho_2[pauli])<eps, print('Problem with qubit',j,':',rho)
        else:
            assert abs(rho[pauli]-initial_rho[pauli])<eps, print('Problem with qubit',j,':',rho)

set_bloch called for qubit 2 with target_expect: {'X': 1.0}, fraction: 1, update: True
Current Bloch vector for qubit 2: {'X': np.float64(-0.00164962398513394), 'Y': np.float64(0.0018514107167697641), 'Z': np.float64(0.9941432756951121)}
Target Bloch vector for qubit 2: {'X': 1.0, 'Y': 0, 'Z': 0}
Computed unitary matrix U for qubit 2:
[[ 0.70651957+0.00065843j -0.7076929 -0.00065843j]
 [ 0.7076929 -0.00065843j  0.70651957-0.00065843j]]
Fractional unitary matrix U for qubit 2 (fraction=1):
[[ 0.70651957+0.00065843j -0.7076929 -0.00065843j]
 [ 0.7076929 -0.00065843j  0.70651957-0.00065843j]]
Euler angles for qubit 2: theta=1.5724556646899477, phi=-0.0018623156456962957, lambda=-1.5451062628136928e-06
Tomography updated after applying set_bloch.
Qubit 2 Bloch vector after set_bloch: {'X': np.float64(0.9932035763499144), 'Y': np.float64(-0.0009542455786500902), 'Z': np.float64(0.00021842146717396922)}


for j in range(n):
    print(f"Qubit {j} Bloch vector: {graph.get_bloch(j)}")

The relationship between qubit 2 and any other should now have $\langle ZX \rangle = 1$ (or $\langle XZ \rangle = 1$ depending on order. Check that this is true.

In [9]:
for j in range(n):
    if j!=2:
        assert abs(graph.get_relationship(2,j)['XZ']-1)<eps
        assert abs(graph.get_relationship(j,2)['ZX']-1)<eps

# for j in range(n):
#     if j != 2:
#         # Print the relationship values for debugging
#         rel_2j = graph.get_relationship(2, j)
#         rel_j2 = graph.get_relationship(j, 2)
#         print(f"Relationship (2, {j}): {rel_2j}")
#         print(f"Relationship ({j}, 2): {rel_j2}")
        
#         # Assert the relationships
#         assert abs(rel_2j['XZ'] - 1) < eps, f"Problem with relationship (2, {j}): {rel_2j}"
#         assert abs(rel_j2['ZX'] - 1) < eps, f"Problem with relationship ({j}, 2): {rel_j2}"
# for j in range(n):
#     if j != 2:
#         bloch2 = graph.get_bloch(2)
#         blochj = graph.get_bloch(j)
#         rel = graph.get_relationship(2, j)
#         expected = bloch2['X'] * blochj['Z']
#         print(f"Relationship (2,{j}):", rel)
#         assert abs(rel.get('XZ', 0.0) - expected) < eps

Set qubit 1 to the $\langle Y \rangle=1$ state (without updating the tomography), and then set qubit 3 to the $\langle X \rangle = \langle Z \rangle$ and $\langle Z \rangle=0$ state (with the update). Check that they are correct, and that the others are still what they should be too.

In [10]:
rho_1 = {'X': 0.0, 'Y': 1.0, 'Z': 0.0}
rho_3 = {'X': 0.0, 'Y': 1.0, 'Z': 0.0}
graph.set_bloch(rho_1, 1, update=False)
graph.set_bloch(rho_3, 3)
rho = graph.get_bloch(3)
print(f"Qubit 3 Bloch vector after set_bloch: {rho}")

rho_3 = {'X': 1/np.sqrt(2), 'Y': 1/np.sqrt(2), 'Z': 0.0}
for j in range(n):
    rho = graph.get_bloch(j)
    for pauli in ['X','Y','Z']:
        if j==1:
            assert abs(rho[pauli]-rho_1[pauli])<eps, print('Problem with qubit',j,':',rho)
        elif j==2:
            assert abs(rho[pauli]-rho_2[pauli])<eps, print('Problem with qubit',j,':',rho)
        elif j==3:
            print(f"Qubit {j} - Actual: {rho[pauli]}, Expected: {rho_3[pauli]}")
            assert abs(rho[pauli]-rho_3[pauli])<eps, print('Problem with qubit',j,':',rho)
        else:
            assert abs(rho[pauli]-initial_rho[pauli])<eps, print('Problem with qubit',j,':',rho)

set_bloch called for qubit 1 with target_expect: {'X': 0.0, 'Y': 1.0, 'Z': 0.0}, fraction: 1, update: False
Current Bloch vector for qubit 1: {'X': np.float64(0.000563652158710542), 'Y': np.float64(0.0005809653634291238), 'Z': np.float64(0.9927901603756323)}
Target Bloch vector for qubit 1: {'X': 0.0, 'Y': 1.0, 'Z': 0.0}
Computed unitary matrix U for qubit 1:
[[ 7.07313616e-01-2.00728301e-04j  2.00728301e-04+7.06899829e-01j]
 [-2.00728301e-04+7.06899829e-01j  7.07313616e-01+2.00728301e-04j]]
Fractional unitary matrix U for qubit 1 (fraction=1):
[[ 7.07313616e-01-2.00728301e-04j  2.00728301e-04+7.06899829e-01j]
 [-2.00728301e-04+7.06899829e-01j  7.07313616e-01+2.00728301e-04j]]
Euler angles for qubit 1: theta=1.5702111425066336, phi=1.5713640722466151, lambda=-1.5707964929127647
set_bloch called for qubit 3 with target_expect: {'X': 0.0, 'Y': 1.0, 'Z': 0.0}, fraction: 1, update: True
Current Bloch vector for qubit 3: {'X': np.float64(-0.0024868410762098657), 'Y': np.float64(-0.001464968

Do half of the rotation of qubit 4 from the $\langle Z \rangle=1$ state to $\langle X \rangle=-1$.

In [11]:
rho_4 = {'X': -1.0, 'Y': 0.0, 'Z': 0.0}
graph.set_bloch(rho_4,4,fraction=0.5)
rho_4 = {'X': -1/np.sqrt(2), 'Y': 0.0, 'Z': 1/np.sqrt(2)}
for j in range(n):
    rho = graph.get_bloch(j)
    for pauli in ['X','Y','Z']:
        if j==1:
            assert abs(rho[pauli]-rho_1[pauli])<eps, print('Problem with qubit',j,':',rho)
        elif j==2:
            assert abs(rho[pauli]-rho_2[pauli])<eps, print('Problem with qubit',j,':',rho)
        elif j==3:
            assert abs(rho[pauli]-rho_3[pauli])<eps, print('Problem with qubit',j,':',rho)
        elif j==4:
            assert abs(rho[pauli]-rho_4[pauli])<eps, print('Problem with qubit',j,':',rho)

set_bloch called for qubit 4 with target_expect: {'X': -1.0, 'Y': 0.0, 'Z': 0.0}, fraction: 0.5, update: True
Current Bloch vector for qubit 4: {'X': np.float64(-0.001255404548125311), 'Y': np.float64(-7.190346346629909e-05), 'Z': np.float64(0.993148289084464)}
Target Bloch vector for qubit 4: {'X': -1.0, 'Y': 0.0, 'Z': 0.0}
Computed unitary matrix U for qubit 4:
[[ 0.70755355+2.55970818e-05j  0.70665973+2.55970818e-05j]
 [-0.70665973+2.55970818e-05j  0.70755355-2.55970818e-05j]]
Fractional unitary matrix U for qubit 4 (fraction=0.5):
[[ 0.92400042+1.38512285e-05j  0.38239145+1.38512285e-05j]
 [-0.38239145+1.38512285e-05j  0.92400042-1.38512285e-05j]]
Euler angles for qubit 4: theta=0.7847661310680109, phi=3.1415414404482314, lambda=-3.141571421448553
Tomography updated after applying set_bloch.


Various tests for two qubits.

In [12]:
relationships = {'ZZ':+1}
graph.set_relationship(relationships,1,2)
rho_12 = graph.get_relationship(1,2)
for (pauli,sign) in relationships.items():
    assert abs(rho_12[pauli]-sign)<2*eps, print('Problem with `set_relationship`:',rho_12)

relationships = {'XX':+1}
graph.set_relationship(relationships,1,2)
rho_12 = graph.get_relationship(1,2)
for (pauli,sign) in relationships.items():
    assert abs(rho_12[pauli]-sign)<2*eps, print('Problem with `set_relationship`:',rho_12)
    
relationships = {'ZX':+1,'XZ':+1}
graph.set_relationship(relationships,1,2)
rho_12 = graph.get_relationship(1,2)
for (pauli,sign) in relationships.items():
    assert abs(rho_12[pauli]-sign)<2*eps, print('Problem with `set_relationship`:',rho_12)